#### 1. CPU使用负载国道的查询步骤
1. 4个步骤进行排查   

```bash
1. top oder by with P：1040   # 首先按进程负载排序找到  axLoad(pid)
2. top -Hp 进程PID：1073       # 找到相关负载 线程PID
3. printf “0x%x\n”线程PID： 0x431   # 将线程PID转换为 16进制，为后面查找 jstack 日志做准备
4. jstack  进程PID | vim +/十六进制线程PID -        # 例如：jstack 1040|vim +/0x431 -  ; 定位到代码行
```

2. top命令
    * 关于load average: 一定程度上表示cpu负载, top分别输出1分钟,5分钟, 15分钟的平均坤负载情况  
       但是linux下负载不仅记录了running状态的进程, 还记录了sleep状态的进程; 前者占用cpu, 后者往往是因为io阻塞. 所以该数值在linux下不能100%表示cpu的负载  
    * -P : 按照cpu使用排序
    * -M : 按照内存使用排序